In [1]:
import pandas as pd
import numpy as np
import json

In [25]:
import datetime
from google.oauth2 import service_account
from googleapiclient import discovery

creds_path = "/Users/haozhang/creds/gcp.json"
SCOPES = ['https://www.googleapis.com/auth/monitoring.read']
credentials = service_account.Credentials.from_service_account_file(creds_path, scopes=SCOPES)
monitoring = discovery.build('monitoring', 'v3', credentials=credentials).projects().timeSeries()

nowTime = datetime.datetime.utcnow()
ST = (nowTime - datetime.timedelta(minutes = 15)).isoformat()+'Z'
ET = (nowTime - datetime.timedelta(minutes = 5)).isoformat()+'Z'

In [132]:
def fetch(service,metric_filter,crossSeriesReducer,perSeriesAligner,groupByFields,view='FULL',alignment_period='600s',pageSize='10000'):
    response = service.list(
                       name='projects/alg-hn-insights',
                       filter=f'metric.type = "bigquery.googleapis.com/{metric_filter}"',
                       interval_endTime=ET,
                       interval_startTime=ST,
                       aggregation_alignmentPeriod=alignment_period,
                       aggregation_crossSeriesReducer=crossSeriesReducer,
                       aggregation_perSeriesAligner=perSeriesAligner,
                       aggregation_groupByFields = groupByFields,
                       view = view,
                       pageSize=pageSize).execute()
    
    return response

In [133]:
def process_dic(json):
    dic={}
    p = json.get('points',[])
    dic['startTime'] = [x['interval']['startTime'] for x in p]
    dic['endTime'] = [x['interval']['endTime'] for x in p]
    dic['value'] = [x['value']['int64Value'] for x in p]
    
    dic['subscription'] = json['resource']['type']
    dic['resource_type']= json['resource']['labels']['subscription_id']
    dic['metric']= json['metric']['type']
    dic['operation_type']= json['metric']['labels']['operation_type']
    dic['response_code']= json['metric']['labels']['response_code']
    df = pd.DataFrame(dic)
    return df

In [140]:
nowTime = datetime.datetime.utcnow()
ST = (nowTime - datetime.timedelta(minutes = 1500)).isoformat()+'Z'
ET = (nowTime - datetime.timedelta(minutes = 5)).isoformat()+'Z'
object_count = fetch(monitoring,'query/scanned_bytes_billed','REDUCE_NONE','ALIGN_NONE',[])
total_byte_seconds = fetch(monitoring,'query/count','REDUCE_SUM','ALIGN_SUM',[])
total_bytes = fetch(monitoring,'storage/table_count','REDUCE_SUM','ALIGN_SUM',[])
# sent_bytes_count = fetch(monitoring,'network/sent_bytes_count','REDUCE_MAX','ALIGN_MAX',[])
# received_bytes_count = fetch(monitoring,'network/received_bytes_count','REDUCE_NONE','ALIGN_NONE',[])
# request_count = fetch(monitoring,'api/request_count','REDUCE_MAX','ALIGN_MAX',[])

In [141]:
object_count

{'timeSeries': [{'metric': {'labels': {'priority': 'batch'},
    'type': 'bigquery.googleapis.com/query/scanned_bytes_billed'},
   'resource': {'type': 'global', 'labels': {'project_id': 'alg-hn-insights'}},
   'metricKind': 'DELTA',
   'valueType': 'INT64',
   'points': [{'interval': {'startTime': '2020-07-28T07:17:00Z',
      'endTime': '2020-07-28T07:18:00Z'},
     'value': {'int64Value': '20971520'}},
    {'interval': {'startTime': '2020-07-28T07:16:00Z',
      'endTime': '2020-07-28T07:17:00Z'},
     'value': {'int64Value': '125829120'}},
    {'interval': {'startTime': '2020-07-28T07:04:00Z',
      'endTime': '2020-07-28T07:05:00Z'},
     'value': {'int64Value': '20971520'}},
    {'interval': {'startTime': '2020-07-28T07:02:00Z',
      'endTime': '2020-07-28T07:03:00Z'},
     'value': {'int64Value': '6191841280'}},
    {'interval': {'startTime': '2020-07-28T07:01:00Z',
      'endTime': '2020-07-28T07:02:00Z'},
     'value': {'int64Value': '2226126848'}},
    {'interval': {'startTi

In [142]:
df = pd.concat(map(pd.json_normalize,[object_count['timeSeries'],total_byte_seconds['timeSeries'],total_bytes['timeSeries']]))
# df['ST'] = df.points.apply(lambda x: x[0]['interval']['startTime'])
# df['ET'] = df.points.apply(lambda x: x[0]['interval']['endTime'])
# df['value'] = df.points.apply(lambda x: x[0]['value']['doubleValue'])
# df.drop('points',axis=1)
df

,metricKind,valueType,points,metric.labels.priority,metric.type,resource.type,resource.labels.project_id
0,DELTA,INT64,[{'interval': {'startTime': '2020-07-28T07:17:...,batch,bigquery.googleapis.com/query/scanned_bytes_bi...,global,alg-hn-insights
1,DELTA,INT64,[{'interval': {'startTime': '2020-07-28T07:17:...,interactive,bigquery.googleapis.com/query/scanned_bytes_bi...,global,alg-hn-insights
0,GAUGE,INT64,[{'interval': {'startTime': '2020-07-28T13:14:...,NaN,bigquery.googleapis.com/query/count,bigquery_project,alg-hn-insights
1,GAUGE,INT64,[{'interval': {'startTime': '2020-07-28T13:14:...,NaN,bigquery.googleapis.com/query/count,global,alg-hn-insights
0,GAUGE,INT64,[{'interval': {'startTime': '2020-07-28T10:34:...,NaN,bigquery.googleapis.com/storage/table_count,bigquery_dataset,alg-hn-insights


In [103]:
df_t = pd.concat(map(process_dic,t['timeSeries']))

TypeError: 'float' object is not subscriptable

In [13]:
df_t

,startTime,endTime,value,subscription,resource_type,metric,operation_type,response_code
0,2020-07-28T09:04:36.764907Z,2020-07-28T09:14:36.764907Z,9030423,pubsub_subscription,cdc-secretmatchclub,pubsub.googleapis.com/subscription/byte_cost,ack,success
0,2020-07-28T09:04:36.764907Z,2020-07-28T09:14:36.764907Z,47199320,pubsub_subscription,cdc-secretmatchclub,pubsub.googleapis.com/subscription/byte_cost,pull,success


In [94]:
f=open(r'testjson.json', 'r+')
bytecost= json.load(f)

In [95]:
import time
t = time.time()
pd.json_normalize(bytecost['timeSeries'])
print(time.time()-t)

7.454866170883179


In [297]:
t1 = time.time()
mydict = {}
with open(r'logs.json', 'r+') as f:

    obj = bytecost["timeSeries"]
    for n in range(len(obj)):
        obj1 = obj[n]["points"]
        
        mydict["response_code"]=obj[n]["metric"]["labels"]["response_code"]
        mydict["operation_type"]=obj[n]["metric"]["labels"]["operation_type"]
        mydict["metric"]=obj[n]["metric"]["type"]
        mydict["resource_type"]=obj[n]["resource"]["type"]
        mydict["subscription"]=obj[n]["resource"]["labels"]["subscription_id"]
        
        mydict["startTime"]=obj1[0]["interval"]["startTime"]
        mydict["endTime"]=obj1[0]["interval"]["endTime"]
        mydict["value"]=obj1[0]["value"]["int64Value"]
        f.write('\n' + json.dumps(mydict, separators=(',', ':')))
print(f'time spent {round(time.time()-t1,4)}')

time spent 0.9126


In [295]:
def process(obj):
    p = m.get('points',[])
    dic['startTime'] = [x['interval']['startTime'] for x in p]
    dic['endTime'] = [x['interval']['endTime'] for x in p]
    dic['value'] = [x['value']['int64Value'] for x in p]

    dic['subscription'] = m['resource']['type']
    dic['resource_type']= m['resource']['labels']['subscription_id']
    dic['metric']= m['metric']['type']
    dic['operation_type']= m['metric']['labels']['operation_type']
    dic['response_code']= m['metric']['labels']['response_code']
    return dic

In [308]:
t1 = time.time()
dic = {}
with open(r'logs.json', 'r+') as f:

    obj = bytecost["timeSeries"]
    for n in range(len(obj)):
        p = [m.get('points',[])[0]]
        dic['startTime'] = [x['interval']['startTime'] for x in p]
        dic['endTime'] = [x['interval']['endTime'] for x in p]
        dic['value'] = [x['value']['int64Value'] for x in p]

        dic['subscription'] = m['resource']['type']
        dic['resource_type']= m['resource']['labels']['subscription_id']
        dic['metric']= m['metric']['type']
        dic['operation_type']= m['metric']['labels']['operation_type']
        dic['response_code']= m['metric']['labels']['response_code']
        f.write('\n' + json.dumps(mydict, separators=(',', ':')))
print(f'time spent {round(time.time()-t1,4)}')

time spent 0.9603


In [147]:
response = monitoring.list(
                       name='projects/alg-hn-insights',
                       filter='metric.type = "bigquery.googleapis.com/storage/table_count"',
                       interval_endTime=ET,
                       interval_startTime=ST,
                       pageSize='10000').execute()

response

{'timeSeries': [{'metric': {'type': 'bigquery.googleapis.com/storage/table_count'},
   'resource': {'type': 'bigquery_dataset',
    'labels': {'dataset_id': '196983973', 'project_id': 'alg-hn-insights'}},
   'metricKind': 'GAUGE',
   'valueType': 'INT64',
   'points': [{'interval': {'startTime': '2020-07-28T10:30:00Z',
      'endTime': '2020-07-28T10:30:00Z'},
     'value': {'int64Value': '349'}},
    {'interval': {'startTime': '2020-07-28T10:00:00Z',
      'endTime': '2020-07-28T10:00:00Z'},
     'value': {'int64Value': '349'}},
    {'interval': {'startTime': '2020-07-28T09:30:00Z',
      'endTime': '2020-07-28T09:30:00Z'},
     'value': {'int64Value': '349'}},
    {'interval': {'startTime': '2020-07-28T09:00:00Z',
      'endTime': '2020-07-28T09:00:00Z'},
     'value': {'int64Value': '349'}},
    {'interval': {'startTime': '2020-07-28T08:30:00Z',
      'endTime': '2020-07-28T08:30:00Z'},
     'value': {'int64Value': '349'}},
    {'interval': {'startTime': '2020-07-28T08:00:00Z',
    

{'unit': '1'}